In [ ]:
#WARNING TRACK OPTION NEEDS SIGNIFICANT MEMORY
import sys
import json
import time
import os

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse.linalg import svds
import scipy.sparse
import matplotlib.pyplot as plt
import csv
import operator

In [ ]:
ARTIST_MODE = True
files_to_use = 100

In [ ]:
data_location_desktop = "/home/ubutubiggerbetter/shared/data/spotify_data/data/new_new_formatted_data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data/new_formatted_data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data/tmp_test"
raw_data_location = os.path.join(data_location_desktop, '..')
data_location = data_location_desktop
save_location = data_location
NUM_PLAYLISTS = 1000000
NUM_TRACKS = 2262292
NUM_ARTISTS = 295860 #<- from stats.txt


isdir = os.path.isdir(data_location)
if isdir == False:
    print(data_location, ' does not exist')


In [ ]:
#Helpful commands:
#type() tells type
#sys.getsizeof(obj) tell size in bytes of obj


In [ ]:
def convert_to_full_charmat(artist_count, num_files, data_location, artist_mat = True):
    if artist_mat == True:
        full_mat_csc = csc_matrix((artist_count, 1000 * num_files))
    else:
        full_mat_csc = csc_matrix((artist_count, 1000 * num_files), dtype=np.uint8)
        full_mat_csc = full_mat_csc.tolil()
    inc = 0;
    filenum = 1;
    start_time = time.time()
    for i in range(0, num_files):
        #gen filename:
        if artist_mat == True:
            fname = 'charmat_pt_'+str(i)+'.npz'
            print('Loading ', fname)
            sparse_mat = scipy.sparse.load_npz(os.sep.join((data_location, fname)))
            sparse_mat = sparse_mat.astype(np.uint16)
            print(sparse_mat.shape)
            full_mat_csc[:, inc:(inc+1000)] = sparse_mat
        else:
            fname = 'track_charmat_pt_'+str(i)+'.npz'
            print('Loading ', fname)
            sparse_mat = scipy.sparse.load_npz(os.sep.join((data_location, fname)))
            sparse_mat = sparse_mat.astype(np.uint8)
            full_mat_csc[:, inc:(inc+1000)] = sparse_mat.tolil()

        inc = inc + 1000;
        print('Elapsed Time: ', (time.time() - start_time)/60, ' minutes')
    if artist_mat == True:
        return full_mat_csc
    else:
        full_mat_csc = full_mat_csc.astype(np.float64)
        return full_mat_csc.tocsc()


In [ ]:
#Read individual files into one big char mat
if ARTIST_MODE == True:
    full_char_mat = convert_to_full_charmat(NUM_ARTISTS, files_to_use, data_location)
    scipy.sparse.save_npz(os.sep.join((save_location, 'artist_full_char_mat')), full_char_mat)
else:
    full_char_mat = convert_to_full_charmat(NUM_TRACKS, files_to_use, data_location, False)
    scipy.sparse.save_npz(os.sep.join((save_location, 'track_full_char_mat')), full_char_mat)

In [ ]:
#Perform SVD:
kval = 200
start_time = time.time()
print(full_char_mat.shape)
U, S, VT = svds(full_char_mat, k = kval)
#Save the result:
Usparse = csc_matrix(U)
Ssparse = csc_matrix(S)
VTsparse = csc_matrix(VT)
scipy.sparse.save_npz(os.sep.join((save_location, 'U')), Usparse)
scipy.sparse.save_npz(os.sep.join((save_location, 'S')), Ssparse)
scipy.sparse.save_npz(os.sep.join((save_location, 'VT')), VTsparse)
print('Elapsed: ', time.time() - start_time)
plt.plot(S[::-1])
plt.xlabel('Latent Factors (Genres)')
plt.ylabel('Information (Size of Singular Values)')
plt.show()


In [ ]:
#Flip matrices so that S is in descending order
S_sorted = S[::-1]
n = len(S)
U[:,0:n] = U[:, n-1::-1]
VT[0:n, :] = VT[n-1::-1,:]
print(VT.shape)
#VT is latent_factors x playlists


#if I have a query playlist, it's going to have artists in it giving it information
#if it's SVD'd with the rest of them...

In [ ]:
if ARTIST_MODE == True:
    # Recover Artist INformation
    artist_file = os.sep.join((data_location, "artist_index_new.csv"))
    artist_dict= {}
    art_index_dict = {}
    fobj = open(artist_file, "r")
    w = csv.DictReader(fobj)
    for row in w:
        artist_dict[row['artist_name']] = row['index']
        art_index_dict[row['index']] = row['artist_name']
    fobj.close()  
else:
    #Recover Track ingormation
    track_file = os.sep.join((data_location, "track_index_new.csv"))
    track_dict = {}
    track_index_dict = {}
    fobj = open(track_file, "r")
    w = csv.DictReader(fobj)
    for row in w:
        track_dict[row['track_name']] = row['index']
        track_index_dict[row['index']] = row['track_name']
    fobj.close()

#Recover Playlist Information:
playlist_file = os.sep.join((data_location, "playlistname_index_new.csv"))
playlist_dict= {}
pl_index_dict = {}
fobj = open(playlist_file, "r")
w = csv.DictReader(fobj)
for row in w:
    playlist_dict[row['playlist_name']] = row['index']
    pl_index_dict[row['index']] = row['playlist_name']
fobj.close()

In [ ]:
#Get first column of U [Artist to latent factors]
#This just gives a view of the latent factors that may exists using the track/artists and the
#playlist names to describe them
latent_factor_count = 50
top_values = 10
for i in range(0, latent_factor_count):
    
    rating_latentfactors = U[:, i]
    playlist_latentfactors = VT[i, :]
    #Get max assoiciated values
    #sorted_indices_art = np.argsort(np.abs(artist_latentfactors)
    #sorted_vals_art = np.sort(np.abs(artist_latentfactors))
                                    
    #ascending_indices_art = np.argsort(np.abs(artist_latentfactors))
    #ascending_indices_pl = np.argsort(np.abs(playlist_latentfactors))
    ascending_indices_rat = np.argsort(rating_latentfactors)
    ascending_indices_pl = np.argsort(playlist_latentfactors)
    sorted_indices_rat = ascending_indices_rat[::-1]
    sorted_indices_pl = ascending_indices_pl[::-1]
    
    #sorted_indices_pl = np.argsort(playlist_latentfactors)
    
    #ascending_indices_art = sorted_indices_art[::-1]
    #ascending_indices_pl= sorted_indices_pl[::-1]
    ratings_count = len(sorted_indices_rat)
    #Now, find the associated artists:
    print('\n\nTop', top_values, ' Ratings/Playlist Name for latent factor ', i, ':')
    if ARTIST_MODE == True:
        print('Best Artists:')
    else:
        print('Best Tracks: ')
    for j in range(0, top_values):
        if ARTIST_MODE == True:
            artist = art_index_dict[str(sorted_indices_rat[j])]
            print(artist, end=', ')
        else:
            track = track_index_dict[str(sorted_indices_rat[j])]
            print(track, end=', ')
            
    print('\nAssociated Names: ')
    for j in range(0, top_values):
        playlist_name = pl_index_dict[str(sorted_indices_pl[j])]
        print(playlist_name, end=', ')

In [ ]:
def recommend_on_playlist(playlist_contents, query_playlist_index, pl_index_dict, \
                          art_index_dict, VT, full_char_mat, latentfactors_to_use = 3, \
                          playlists_to_use = 20, num_recommendations=20):
    print('playlist_name: ', pl_index_dict[str(query_playlist_index)])
    #get the max latent factors index assocaited with the query
    asc_sorted_indices_pl = np.argsort(VT[:,query_playlist_index])
    sorted_indices_pl = asc_sorted_indices_pl[::-1]

    #Use the sorted_indices_pl to map into applicable artists (U)
    recommended_artists = {}
    playlists_used = {}
    MAX_PLAYLISTS = VT.shape[1]
    k = 0
    full_char_mat = full_char_mat.tolil()
    for i in range(0, latentfactors_to_use):
        print(i, ' out of ', latentfactors_to_use)
        #Get the max latent factor index
        latent_factor_i = sorted_indices_pl[i]
        #Get all the playlists' latent factor for this index (corresponds to a given genre)
        latent_factors_v_playlists = VT[latent_factor_i, :]
        #Get the largest values which will be largest latent factors corresponding
        #to this genre
        asc_similar_playlist_indices = np.argsort(latent_factors_v_playlists)
        similar_playlist_indices = asc_similar_playlist_indices[::-1]
    
        playlists_to_use_loop = playlists_to_use
        k = 0
        while k < playlists_to_use_loop:
            #Make sure it's not the query playlist:
            if similar_playlist_indices[k] != query_playlist_index and str(similar_playlist_indices[k]) not in playlists_used:
                print('Similar Playlist is: ', pl_index_dict[str(similar_playlist_indices[k])])
                #Add to the dictionary (so we don't redo)
                playlists_used[str(similar_playlist_indices[k])] = pl_index_dict[str(similar_playlist_indices[k])]
                #get the index; this will be the most similar playlist for the largest latent
                #factor of the query playlist (for now)
                similar_playlist_index = similar_playlist_indices[k]
                #Map the index into original charmat to get similar artists:
                playlists_artists_indices = full_char_mat[:, similar_playlist_index]
            
                #Tally the most similar artists
                non_zero_artist_indices = playlists_artists_indices.nonzero()
                for j in non_zero_artist_indices[0]:
                    nz_index = j
                    if art_index_dict[str(nz_index)] in recommended_artists:
                        recommended_artists[art_index_dict[str(nz_index)]] = recommended_artists[art_index_dict[str(nz_index)]] + 1
                    else:
                        recommended_artists[art_index_dict[str(nz_index)]] = 1
            else:
                if playlists_to_use_loop + 1 < MAX_PLAYLISTS:
                    playlists_to_use_loop = playlists_to_use_loop + 1
            k = k+1
    
    sorted_dict = sorted(recommended_artists.items(), key=operator.itemgetter(1), reverse=True)

    top_recommended_artists = []
    

    #Just go through and compare to queried playlist, as long as item is not
    #already in, then add it (note: with artists this may be dicey....)
    i = 0
    for item, value in sorted_dict:
        if item not in playlist_contents:
            if i >= num_recommendations:
                break
            top_recommended_artists.append(item)
            i = i+1

    return top_recommended_artists

In [ ]:
def playlist_lookup(json_locations, playlist_index):
    #Can construct the filename based on the number of the playlist index 
    #also assuming 0 indexed, so
    file_start = 'mpd.slice.'
    file_end = '.json'
    candidate_val = playlist_index/1000
    print(candidate_val)

        
    from_num = int(np.floor(candidate_val)) * 1000
    to_num = int(np.ceil(candidate_val)) * 1000 -1
    if candidate_val == np.ceil(candidate_val):
        to_num = int(np.ceil(candidate_val + 1)) * 1000 - 1
    file_mid = str(from_num) + '-' + str(to_num)
    fname = file_start + file_mid + file_end
    
    
    print('Loading ', fname)
    f = open(os.path.join(json_locations, fname))
    #Get the json reader:
    js = f.read()
    f.close()
    mpd_slice = json.loads(js)
    q_playlist = mpd_slice['playlists'][playlist_index-from_num]
    tracks = set()
    artists = set()
    for track in q_playlist['tracks']:
        tracks.add(track['track_name'])
        artists.add(track['artist_name'])

    return artists, tracks

In [ ]:
playlist_to_test = 50

latent_factors_used = 4
playlists_referenced = 7
desired_recommendations = 20
start_time = time.time()
artists, tracks = playlist_lookup(raw_data_location, playlist_to_test)
if ARTIST_MODE == True:
    recommendations = recommend_on_playlist(artists, playlist_to_test, pl_index_dict, \
                                            art_index_dict, VT, full_char_mat, \
                                            latent_factors_used, playlists_referenced, \
                                            desired_recommendations)
else:
    recommendations = recommend_on_playlist(tracks, playlist_to_test, pl_index_dict, \
                                            track_index_dict, VT, full_char_mat, \
                                            latent_factors_used, playlists_referenced, \
                                            desired_recommendations)
print('Elapsed: ', time.time() - start_time)
print('Recommendations: ', recommendations)
print('\nOriginal Tracks: ', tracks)
print('\nOriginal Artists:', artists)